# 3D Heating/Cooling Table
Indexed by density, (redshift, metallicity, etc.), and temperature.
Ignores tabulated Hydrogen/Helium. 

You can also read in tables which don't have redshift, instead providing e.g. metallicity, like cloudy_metals_2008_3D.h5.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl
import h5py
import ipywidgets as widgets

In [ ]:
filename = "../../../grackle/grackle_data_files/input/CloudyData_UVB=HM2012_shielded.h5"  # Grackle data for examples
filename = "../../../grackle/grackle_data_files/input/cloudy_metals_2008_3D.h5"

filein = h5py.File(filename, "r")

# Could, at some point, add drop down selectors for primordial/metal heating/cooling
_, parameter1_arr, parameter1_name, parameter2_arr, parameter2_name, _, temperature_arr = filein["CoolingRates/Metals/Cooling"].attrs.values()
temperature_arr = np.log10(temperature_arr)

parameter2_slider = widgets.SelectionSlider(options=parameter2_arr, value=0, description=parameter2_name.decode())

# Create the plot
extent = (parameter1_arr[0], parameter1_arr[-1], temperature_arr[0], temperature_arr[-1])
im = plt.imshow(
    filein["CoolingRates/Metals/Cooling"][:, parameter2_slider.index, :].T,
    norm="log",
    vmin=1e-30,
    vmax=1e-21,
    origin="lower",
    extent=extent,
    interpolation='None'
    )

plt.ylabel("log Temperature")
plt.xlabel(f"log {parameter1_name.decode()}")
plt.colorbar(im)
plt.title(f"Cooling (function/rate?)")

# Add interactivity
# We could use widgets.interact, but this way means we don't need to search for the value, and can instead take the index directly
# It _looks_ worse, but in theory should be slightly more perfomative

def set_data(change):
    im.set_data(
       filein["CoolingRates/Metals/Cooling"][:, parameter2_slider.index, :].T
    )

parameter2_slider.observe(set_data)

display(parameter2_slider)

plt.show()

In [ ]:
filein.close()
plt.close()